# ML Parsing #
Initial parsing of the grade submissions. It is meant only for research purposes and not for production environments.

_What it does?_
- Parses the intial csv files (form csv + grades uploaded in csv format)
- Shows basic info of the data
- Connects form submissions with their associated grades and cleans entry with no grades
- Parses the submission for the CSD Department
- Drops invalid values and fixes any other issues with the dataset
- Produces a final dataset with the form submissions and the grades. Each course found in the dataset is represented as a column. Only the valid courses are kept as found by the OPEN API of AUTH. The dataset contains 89 courses. 

*Some columns refer to the same course but they are with different codes (old/new study program)
- This is fixed, old courses are assigned to new course codes.

**ATTENTION!!: Some outputs have been deleted for privacy reasons. PLEASE DO NOT UPLOAD OUTPUTS WITH PERSONAL INFO** 

In [1]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [2]:
#Define files
form_csv = "Form_Responses_11_09.csv"
folder_with_grade_csvs = "../GradeParser/csv/"
# Json with the valid courses as got from OPEN AUTH API (https://ws-ext.it.auth.gr/)
valid_courses_json = "courses_ids_600000014.json"

## General Parsing ##

In [ ]:
#reading the form submissions (download csv file)
submissions_csv = pd.read_csv(form_csv)
display(submissions_csv)

In [4]:
submissions_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 16 columns):
Timestamp                                                                   64 non-null object
Ηλικία                                                                      64 non-null int64
Φύλο                                                                        64 non-null object
Σχολή Φοίτησης                                                              64 non-null object
Επέλεξα τη σχολή μου διότι:                                                 64 non-null object
Κατά μέσο όρο την εβδομάδα, διαβάζω:                                        64 non-null object
Μέσα στο εξάμηνο, παρακαλουθώ:                                              64 non-null object
Υπήρξε ανάγκη για φροντηστηριακή βοήθεια σε κάποιο μάθημα έως τώρα;         64 non-null object
Μετά το πτυχίο, θα ήθελα να ακολουθήσω:                                     64 non-null object
Ποιο από τα παρακάτω ισχύει;               

In [5]:
#Create an extract column to store the paths of the csvs with the grades
csv = [None] * (submissions_csv.shape[0])
submissions_csv["CSV"] = pd.Series(csv).to_numpy()
# display(submissions_csv)
# print(submissions_csv['Filename'])

In [ ]:
#Find associated csv
for r,d,f in os.walk(folder_with_grade_csvs):
    for file in f:
        if ".csv" in file: # For each csv
            without_ext = file.replace("_results.csv","") #Remove extensions added by the parser\n",
            # Set the csv found \n",
            submissions_csv.loc[submissions_csv["Filename"].str.contains(without_ext,regex=False),"CSV"] = r + file
display(submissions_csv)

In [ ]:
# Nan CSV entries
display(submissions_csv[submissions_csv['CSV'].isnull()])
# Drop those entries with no csv associated
submissions_csv.dropna(subset=['CSV'],inplace = True)
display(submissions_csv)

## Parsing for Departments ##

In [8]:
# Check unique departments
display(submissions_csv.groupby('Σχολή Φοίτησης')['Timestamp'].nunique())


# Get the CSD department
csd = submissions_csv[submissions_csv['Σχολή Φοίτησης'] == "Πληροφορική ΑΠΘ"]
# hmmy = submissions_csv[submissions_csv['Σχολή Φοίτησης'] == "ΗΜΜΥ ΑΠΘ"]


Σχολή Φοίτησης
Αρχιτεκτονική ΑΠΘ             1
Γεωλογία ΑΠΘ                  1
Δημοσιογραφία & ΜΜΕ,ΑΠΘ       1
ΗΜΜΥ ΑΠΘ                     12
Θεολογική ΑΠΘ                 1
Μηχανολογων Μηχανικων ΑΠΘ     1
Οικονομικών Επιστημών ΑΠΘ     1
Πληροφορική ΑΠΘ              43
Πολιτικών Μηχανικών ΑΠΘ       1
Name: Timestamp, dtype: int64

### CSD Department - Grade Statistics ###

In [9]:
# Concat all grades (all the grade csvs together)
csd_csv_files = csd['CSV'].to_numpy()
li = []
# pd.read_csv(csd_csv_files[5],encoding='utf-8')
colnames = ["C_ID","C_NAME","TYPE","EXAM_YEAR","EXAM_PERIOD","ECTS","DM","FACTOR","GRADE"]
# coltypes = {"C_ID":str,"C_NAME":str,"TYPE":str,"EXAM_YEAR":str,"EXAM_PERIOD":str,"ECTS":np.int32,"DM":np.int32,"FACTOR":np.int32,"GRADE":np.float64}
for csv_file in csd_csv_files:
    f = open(csv_file,encoding='utf-8')
    df = pd.read_csv(f, index_col=None, header=None, names = colnames)
    li.append(df)

csd_grades = pd.concat(li, axis=0, ignore_index=True)
display(csd_grades)

,C_ID,C_NAME,TYPE,EXAM_YEAR,EXAM_PERIOD,ECTS,DM,FACTOR,GRADE
0,NCO-01-01,ΜΑΘΗΜΑΤΙΚΗ ΑΝΑΛΥΣΗ Ι,Υ,2017 - 2018,ΦΕΒΡ,5.0,4.0,5.0,10
1,NCO-01-02,ΕΙΣΑΓΩΓΗ ΣΤΗΝ ΠΛΗΡΟΦΟΡΙΚΗ,Υ,2017 - 2018,ΦΕΒΡ,4.0,3.0,4.0,10
2,NCO-01-03,ΓΡΑΜΜΙΚΗ ΑΛΓΕΒΡΑ,Υ,2017 - 2018,ΦΕΒΡ,5.0,4.0,5.0,10
3,NCO-01-04,ΔΙΑΚΡΙΤΑ ΜΑΘΗΜΑΤΙΚΑ,Υ,2017 - 2018,ΦΕΒΡ,7.0,5.0,7.0,9
4,NCO-01-05,ΒΑΣΙΚΕΣ ΑΡΧΕΣ ΠΡΟΓΡΑΜΜΑΤΙΣΜΟΥ,Υ,2017 - 2018,ΦΕΒΡ,9.0,7.0,9.0,10
...,...,...,...,...,...,...,...,...,...
1022,NCO-04-01,ΣΧΕΔΙΑΣΗ ΓΛΩΣΣΩΝ ΠΡΟΓΡΑΜΜΑΤΙΣΜΟΥ,Υ,2018 - 2019,ΙΟΥΝ,7.0,5.0,7.0,6
1023,NCO-04-02,ΤΕΧΝΗΤΗ ΝΟΗΜΟΣΥΝΗ,Υ,2018 - 2019,ΙΟΥΝ,5.5,4.0,5.5,8
1024,NCO-04-03,ΑΛΓΟΡΙΘΜΟΙ,Υ,,,7.0,6.0,7.0,NaN
1025,NCO-04-04,ΕΠΙΚΟΙΝΩΝΙΑ ΑΝΘΡΩΠΟΥ-ΥΠΟΛΟΓΙΣΤΗ,Υ,2018 - 2019,ΙΟΥΝ,5.5,4.0,5.5,7.5


In [10]:
# Find more information on the courses
unique_courses = csd_grades["C_ID"].unique()
print("Total unique courses:{}".format(len(unique_courses)))
csd_grades['C_ID'] = csd_grades['C_ID'].str.replace(' ','',regex=True)
csd_grades["C_ID"].value_counts()

Total unique courses:101


NCO-01-01    41
NCO-02-01    41
NCO-02-05    41
NCO-02-03    41
NCO-02-02    40
             ..
NNA-07-02     1
NDM-06-03     1
NDM-06-04     1
NGE-08-02     1
CO-04-02      1
Name: C_ID, Length: 101, dtype: int64

In [11]:
# Fixing Issues
def fix_issues(df):
    
    df['C_ID'] = df['C_ID'].str.replace(' ','',regex=True) # Removing blank cases
    
    df['C_NAME'] = df['C_NAME'].str.replace('*','',regex=True) # Removing stars
    df['C_NAME'] = df['C_NAME'].str.strip() # Removing leading and trailing spaces

    # Fixing issue with parser
    df['C_NAME'] = df['C_NAME'].replace('ΑΝΤΙΚΕΙΜΕΝΟΣΤΡΕΦΗΣ','ΑΝΤΙΚΕΙΜΕΝΟΣΤΡΕΦΗΣ ΠΡΟΓΡΑΜΜΑΤΙΣΜΟΣ',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΘΕΩΡΙΑ ΓΛΩΣΣΩΝ ΚΑΙ','ΘΕΩΡΙΑ ΓΛΩΣΣΩΝ ΚΑΙ ΑΥΤΟΜΑΤΩΝ',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΤΕΧΝΙΚΕΣ ΔΟΜΗΣΗΣ','ΤΕΧΝΙΚΕΣ ΔΟΜΗΣΗΣ ΔΕΔΟΜΕΝΩΝ',regex=False)
    
    # Grouping old courses with new ones with the same name - CHANGED TO MANUALLY
    unique_courses = df['C_NAME'].unique()
    for course in unique_courses:
        course_grades = df[df['C_NAME'] == course]
        if course_grades['C_ID'].nunique() > 1:
            course_codes = course_grades['C_ID'].value_counts().index.values
            for i in range(1,len(course_codes)):
                df['C_ID'] = df['C_ID'].replace(course_codes[i],course_codes[0],regex=False)
                
                
    ## Manual Work ## 
    df['C_NAME'] = df['C_NAME'].replace('ΑΝΑΓΝΩΡΙΣΗ ΠΡΟΤΥΠΩΝ','ΑΝΑΓΝΩΡΙΣΗ ΠΡΟΤΥΠΩΝ -ΣΤΑΤΙΣΤΙΚΗ ΜΑΘΗΣΗ',regex=False)
    df['C_ID'] = df['C_ID'].replace('NDM-06-03','NDM-06-04',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΑΝΩΤΕΡΑ ΜΑΘΗΜΑΤΙΚΑ Ι','ΜΑΘΗΜΑΤΙΚΗ ΑΝΑΛΥΣΗ Ι',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-01-01','NCO-01-01',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΑΝΩΤΕΡΑ ΜΑΘΗΜΑΤΙΚΑ ΙΙ','ΜΑΘΗΜΑΤΙΚΗ ΑΝΑΛΥΣΗ ΙΙ',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-02-01','NCO-02-01',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΕΙΣΑΓΩΓΗ ΣΤΟΥΣ ΥΠΟΛΟΓΙΣΤΕΣ','ΕΙΣΑΓΩΓΗ ΣΤΗΝ ΠΛΗΡΟΦΟΡΙΚΗ',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-01-02','NCO-01-02',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΘΕΩΡΙΑ ΓΛΩΣΣΩΝ ΚΑΙ ΑΥΤΟΜΑΤΩΝ','ΘΕΩΡΙΑ ΥΠΟΛΟΓΙΣΜΟΥ',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-02-04','NCO-02-05',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΝΕΥΡΩΝΙΚΑ ΔΙΚΤΥΑ','ΝΕΥΡΩΝΙΚΑ ΔΙΚΤΥΑ - ΒΑΘΙΑ ΜΑΘΗΣΗ',regex=False)
    df['C_ID'] = df['C_ID'].replace('NDM-07-03','NDM-07-05',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΠΙΘΑΝΟΤΗΤΕΣ KAI ΣΤΑΤΙΣΤΙΚΗ','ΠΙΘΑΝΟΤΗΤΕΣ & ΣΤΑΤΙΣΤΙΚΗ',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-02-02','NCO-02-02',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΣΧΕΔΙΑΣΗ ΑΛΓΟΡΙΘΜΩΝ','ΑΛΓΟΡΙΘΜΟΙ',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-04-05','NCO-04-03',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΣΧΕΔΙΑΣΗ ΓΛΩΣΣΩΝ ΚΑΙ ΜΕΤΑΓΛΩΤΤΙΣΤΕΣ','ΣΧΕΔΙΑΣΗ ΓΛΩΣΣΩΝ ΠΡΟΓΡΑΜΜΑΤΙΣΜΟΥ ',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-04-03','NCO-04-01',regex=False)
    
    # More manual
    df['C_ID'] = df['C_ID'].replace('CO-01-05','NCO-01-05',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-01-06','NCO-01-04',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-01-03','NCO-01-03',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-02-05','NCO-02-04',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-04-02','NCO-04-05',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-03-03','NCO-03-04',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-04-04','NCO-04-02',regex=False)
    
    return df

In [12]:
# Printing the full dataframe
def print_full(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df)

In [13]:
# Checking more info
unique_names = csd_grades.groupby('C_NAME')['C_ID'].value_counts()
print_full(unique_names)
print("Total:" + str(csd_grades['C_NAME'].nunique()))


C_NAME                                                      C_ID     
 COMPUTER GRAPHICS -ΓΡΑΦΙΚΑ                                 ER-01-02      1
 DISTRIBUTED SYSTEMS -ΚΑΤΑΝΕΜΗΜΕΝΑ ΔΙΑΔΙΚΤΥΑΚΑ ΣΥΣΤΗΜΑΤΑ    ER-01-01      1
 ERASMUS 1                                                  ER-01-01      1
 ERASMUS 2                                                  ER-01-02      1
 INTELLIGENT SYSTEMS                                        ER-01-03      1
 INTRODUCTION TO SWEDISH LEVEL 1                            ER-01-02      1
 MATHEMATICAL STATISTICS                                    ER-01-01      1
 SOFTWARE DEVELOPMENT FOR REAL-TIME SYSTEMS                 ER-01-04      1
 ΑΓΓΛΙΚΑ Ι  *                                               CO-01-07      4
 ΑΓΓΛΙΚΑ ΙV                                                 CO-04-07      1
 ΑΓΓΛΙΚΑ ΙΙ                                                 CO-02-07      1
 ΑΓΓΛΙΚΑ ΙΙ  *                                              CO-02-07      3
 ΑΓΓΛΙΚΑ ΙΙΙ  *   

In [14]:
csd_grades = fix_issues(csd_grades)

In [15]:
unique_names = csd_grades.groupby('C_NAME')['C_ID'].value_counts()
print_full(unique_names)
print("Total:" + str(csd_grades['C_NAME'].nunique()))

C_NAME                                                   C_ID     
COMPUTER GRAPHICS -ΓΡΑΦΙΚΑ                               ER-01-02      1
DISTRIBUTED SYSTEMS -ΚΑΤΑΝΕΜΗΜΕΝΑ ΔΙΑΔΙΚΤΥΑΚΑ ΣΥΣΤΗΜΑΤΑ  ER-01-01      1
ERASMUS 1                                                ER-01-01      1
ERASMUS 2                                                ER-01-02      1
INTELLIGENT SYSTEMS                                      ER-01-03      1
INTRODUCTION TO SWEDISH LEVEL 1                          ER-01-02      1
MATHEMATICAL STATISTICS                                  ER-01-01      1
SOFTWARE DEVELOPMENT FOR REAL-TIME SYSTEMS               ER-01-04      1
ΑΓΓΛΙΚΑ Ι                                                CO-01-07      4
ΑΓΓΛΙΚΑ ΙV                                               CO-04-07      1
ΑΓΓΛΙΚΑ ΙΙ                                               CO-02-07      4
ΑΓΓΛΙΚΑ ΙΙΙ                                              CO-03-07      1
ΑΛΓΟΡΙΘΜΟΙ                                               

In [16]:
# Get's the real courses offered as got from the AUTH Open Data API (https://ws-ext.it.auth.gr/)
def get_valid_courses():
    with open(valid_courses_json) as json_file:
        data = json.load(json_file)
        df = pd.DataFrame(data['courses'])
    json_file.close()
    return df['coursecode'].values

In [17]:
import numpy as np
colnames = ["C_ID","C_NAME","TYPE","EXAM_YEAR","EXAM_PERIOD","ECTS","DM","FACTOR","GRADE"]
unique_courses = csd_grades["C_ID"].unique()
course_dictionary = dict()
# Create a dictionary for each course, each row represents a student
for course in get_valid_courses():
    course_dictionary[course] = [-1] * len(csd_csv_files) 
for i in range(0,len(csd_csv_files)):
    csv_file = csd_csv_files[i]
    f = open(csv_file,encoding='utf-8')
    df = pd.read_csv(f, index_col=None, header=None, names = colnames)
    df = fix_issues(df)
    df_courses = df["C_ID"].to_numpy()
    for course_id in df_courses:
        if course_id in course_dictionary:
            course_dictionary[course_id][i] =  df[df["C_ID"] == course_id]["GRADE"].to_numpy()[0]
        else:
            print("Not valid course:" + course_id + '-' + csd_grades[csd_grades["C_ID"] == course_id]['C_NAME'].values)
csd_courses_df = pd.DataFrame.from_dict(course_dictionary)
display(csd_courses_df)

['Not valid course:CO-01-07-ΑΓΓΛΙΚΑ Ι'
 'Not valid course:CO-01-07-ΑΓΓΛΙΚΑ Ι'
 'Not valid course:CO-01-07-ΑΓΓΛΙΚΑ Ι'
 'Not valid course:CO-01-07-ΑΓΓΛΙΚΑ Ι']
['Not valid course:CO-01-04-ΦΥΣΙΚΗ' 'Not valid course:CO-01-04-ΦΥΣΙΚΗ'
 'Not valid course:CO-01-04-ΦΥΣΙΚΗ']
['Not valid course:CO-02-07-ΑΓΓΛΙΚΑ ΙΙ'
 'Not valid course:CO-02-07-ΑΓΓΛΙΚΑ ΙΙ'
 'Not valid course:CO-02-07-ΑΓΓΛΙΚΑ ΙΙ'
 'Not valid course:CO-02-07-ΑΓΓΛΙΚΑ ΙΙ']
['Not valid course:CO-02-06-ΗΛΕΚΤΡΟΝΙΚΗ ΦΥΣΙΚΗ'
 'Not valid course:CO-02-06-ΗΛΕΚΤΡΟΝΙΚΗ ΦΥΣΙΚΗ'
 'Not valid course:CO-02-06-ΗΛΕΚΤΡΟΝΙΚΗ ΦΥΣΙΚΗ']
['Not valid course:CO-02-03-ΠΡΟΗΓΜΕΝΕΣ ΤΕΧΝΙΚΕΣ ΠΡΟΓΡΑΜΜΑΤΙΣΜΟΥ'
 'Not valid course:CO-02-03-ΠΡΟΗΓΜΕΝΕΣ ΤΕΧΝΙΚΕΣ ΠΡΟΓΡΑΜΜΑΤΙΣΜΟΥ'
 'Not valid course:CO-02-03-ΠΡΟΗΓΜΕΝΕΣ ΤΕΧΝΙΚΕΣ ΠΡΟΓΡΑΜΜΑΤΙΣΜΟΥ'
 'Not valid course:CO-02-03-ΠΡΟΗΓΜΕΝΕΣ ΤΕΧΝΙΚΕΣ ΠΡΟΓΡΑΜΜΑΤΙΣΜΟΥ']
['Not valid course:CO-01-07-ΑΓΓΛΙΚΑ Ι'
 'Not valid course:CO-01-07-ΑΓΓΛΙΚΑ Ι'
 'Not valid course:CO-01-07-ΑΓΓΛΙΚΑ Ι'
 'Not valid course:CO-01-07-ΑΓΓΛΙΚΑ Ι']
['Not valid

,NCO-01-01,NCO-01-02,NCO-01-03,NCO-01-04,NCO-01-05,NCO-02-01,NNA-06-01,NNA-06-02,NNA-07-01,NCO-02-02,...,NGE-08-02,NGE-07-03,NGE-06-04,NGE-06-05,NGE-08-03,NNA-08-05,NIS-07-08,NIS-08-04,NDM-06-04,NDM-07-05
0,10,10,10,9,10,8,-1,-1.0,-1,8,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0
1,6,10,7,8,7,8,-1,-1.0,-1,5,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0
2,6,7,5,10,9,6,-1,-1.0,-1,5,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0
3,7,10,8,9,10,6,-1,-1.0,-1,8,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0
4,10,10,10,10,10,10,-1,-1.0,-1,8,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0
5,7,10,6,7,10,7,-1,-1.0,-1,7,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0
6,6,9,9,7,9,7,-1,-1.0,-1,8,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0
7,8,10,10,9,10,10,-1,-1.0,-1,9,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0
8,4,9,4,6,10,7,-1,-1.0,-1,,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0
9,7,10,10,8,10,9,-1,-1.0,-1,9,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0


In [18]:
# Fill nan values with -1
csd_courses_df.fillna(-1,inplace = True)
# Replace blank values with -1
csd_courses_df.replace(r'^\s*$', -1, regex=True,inplace=True)
# Replace ΕΠΙΤ with 10
csd_courses_df.replace(' ΕΠΙΤ',10,inplace=True)
display(csd_courses_df)

,NCO-01-01,NCO-01-02,NCO-01-03,NCO-01-04,NCO-01-05,NCO-02-01,NNA-06-01,NNA-06-02,NNA-07-01,NCO-02-02,...,NGE-08-02,NGE-07-03,NGE-06-04,NGE-06-05,NGE-08-03,NNA-08-05,NIS-07-08,NIS-08-04,NDM-06-04,NDM-07-05
0,10,10,10,9,10,8,-1,-1.0,-1,8,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0
1,6,10,7,8,7,8,-1,-1.0,-1,5,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0
2,6,7,5,10,9,6,-1,-1.0,-1,5,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0
3,7,10,8,9,10,6,-1,-1.0,-1,8,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0
4,10,10,10,10,10,10,-1,-1.0,-1,8,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0
5,7,10,6,7,10,7,-1,-1.0,-1,7,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0
6,6,9,9,7,9,7,-1,-1.0,-1,8,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0
7,8,10,10,9,10,10,-1,-1.0,-1,9,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0
8,4,9,4,6,10,7,-1,-1.0,-1,-1,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0
9,7,10,10,8,10,9,-1,-1.0,-1,9,...,-1.0,-1,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0


In [19]:
# Change to type flaot
csd_courses_df = csd_courses_df.astype(np.float64)

In [20]:
csd_courses_df.describe()

,NCO-01-01,NCO-01-02,NCO-01-03,NCO-01-04,NCO-01-05,NCO-02-01,NNA-06-01,NNA-06-02,NNA-07-01,NCO-02-02,...,NGE-08-02,NGE-07-03,NGE-06-04,NGE-06-05,NGE-08-03,NNA-08-05,NIS-07-08,NIS-08-04,NDM-06-04,NDM-07-05
count,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.0,43.000000,43.0,43.000000,...,43.000000,43.0,43.0,43.0,43.0,43.0,43.000000,43.000000,43.000000,43.000000
mean,6.465116,8.348837,6.720930,6.139535,6.883721,6.395349,-1.0,-0.848837,-1.0,6.744186,...,-0.813953,-1.0,-1.0,-1.0,-1.0,-1.0,-0.883721,-0.813953,-0.767442,-0.604651
std,2.822940,2.147908,3.179739,3.196586,3.831009,3.140667,0.0,0.991241,0.0,2.543367,...,1.219989,0.0,0.0,0.0,0.0,0.0,0.762493,1.219989,1.250471,1.840543
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,...,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000
25%,5.000000,7.500000,5.500000,5.000000,5.500000,5.500000,-1.0,-1.000000,-1.0,6.000000,...,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000
50%,7.000000,9.000000,7.000000,7.000000,8.000000,7.000000,-1.0,-1.000000,-1.0,7.000000,...,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000
75%,8.500000,10.000000,9.000000,8.500000,10.000000,8.500000,-1.0,-1.000000,-1.0,8.000000,...,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,-1.0,5.500000,-1.0,10.000000,...,7.000000,-1.0,-1.0,-1.0,-1.0,-1.0,4.000000,7.000000,7.000000,9.000000


#### Concatenate CSD Student + Grades ####

In [21]:
# Remove unnecessary fields for final dataset
csd_final = csd.drop(['Timestamp', 'Σχολή Φοίτησης',
       'Υποβολή αναλυτικής βαθμολογίας',
       'Συναινώ στη χρήση των στοιχείων μου από την ομάδα του Grade++',
       'Αν θες να λάβεις πρώτος πρόσβαση στην εφαρμογή, συμπλήρωσε το email σου:',
       'Filename', 'CSV'],axis=1)
csd_final.reset_index(drop=True,inplace=True)
display(csd_final)

,Ηλικία,Φύλο,Επέλεξα τη σχολή μου διότι:,"Κατά μέσο όρο την εβδομάδα, διαβάζω:","Μέσα στο εξάμηνο, παρακαλουθώ:",Υπήρξε ανάγκη για φροντηστηριακή βοήθεια σε κάποιο μάθημα έως τώρα;,"Μετά το πτυχίο, θα ήθελα να ακολουθήσω:",Ποιο από τα παρακάτω ισχύει;,Η σχολή απέχει από το σπίτι μου:,Ασχολούμαι εβδομαδιαία με:
0,19,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,Σειρές / Ταινίες
1,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες"
2,22,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,2 - 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,"Αθλητισμό, Σειρές / Ταινίες, Εθελοντισμός"
3,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες"
4,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Εθελοντισμός"
5,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,2 - 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,25 - 45 λεπτά,"Σειρές / Ταινίες, Εθελοντισμός"
6,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Συγκατοικώ με τα αδέρφια μου,10 - 25 λεπτά,"Αθλητισμό, Video Games"
7,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,< 10 λεπτά,"Video Games, Ξένη γλώσσα, Σειρές / Ταινίες"
8,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Σειρές / Ταινίες, Πανεπιστημιακές..."
9,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,10 - 25 λεπτά,"Αθλητισμό, Σειρές / Ταινίες"


In [22]:
# Concat with grades
ldf = [csd_final, csd_courses_df]
csd_final = pd.concat(ldf,axis=1)
display(csd_final)

,Ηλικία,Φύλο,Επέλεξα τη σχολή μου διότι:,"Κατά μέσο όρο την εβδομάδα, διαβάζω:","Μέσα στο εξάμηνο, παρακαλουθώ:",Υπήρξε ανάγκη για φροντηστηριακή βοήθεια σε κάποιο μάθημα έως τώρα;,"Μετά το πτυχίο, θα ήθελα να ακολουθήσω:",Ποιο από τα παρακάτω ισχύει;,Η σχολή απέχει από το σπίτι μου:,Ασχολούμαι εβδομαδιαία με:,...,NGE-08-02,NGE-07-03,NGE-06-04,NGE-06-05,NGE-08-03,NNA-08-05,NIS-07-08,NIS-08-04,NDM-06-04,NDM-07-05
0,19,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,Σειρές / Ταινίες,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,22,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,2 - 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,"Αθλητισμό, Σειρές / Ταινίες, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,2 - 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,25 - 45 λεπτά,"Σειρές / Ταινίες, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Συγκατοικώ με τα αδέρφια μου,10 - 25 λεπτά,"Αθλητισμό, Video Games",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,< 10 λεπτά,"Video Games, Ξένη γλώσσα, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Σειρές / Ταινίες, Πανεπιστημιακές...",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,10 - 25 λεπτά,"Αθλητισμό, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [23]:
# Write to file
csd_final.to_csv("csd.csv",index=False)